```
Image Classification : Implementation in Keras - MNIST Datasets
Published Date: 15 October, 2018

author: Mohammed Innat
email:  innat1994@gmail.com
website: https://iphton.github.io/iphton.github.io/


Please feel free to use and modify this, but keep the above information. Thanks!

```


---

# MNIST Implementation Using Convolutional Network

**About Data:**

The data files `train.csv` and `test.csv` contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is `28` pixels in height and `28` pixels in width, for a total of `784` pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (**train.csv**), has `785` columns. The first column, called `label`, is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

The test data set, (**test.csv**), is the same as the training set, except that it does not contain the `label` column.

Read more on Kaggle: [Digit Recognizer.](https://www.kaggle.com/c/digit-recognizer/data)

---

**Procedure:**
- Data Preprocessing
- Building a Model
- Set Hyper-Parameter
- Evaluate the Model

**Miscellaneous:**
- Save Model and Weight 
- Tensorboard: Visualize the Computational Graph and Parameters.

Finally, Create `csv` file for kaggle submission.

---

**Result:**

I got **Final loss: 0.01153, Final accuracy: 0.99595** with implementing **ConvNet** using TensorFlow high level API **Keras** on **GeForce GTX 1050 Ti**. 

I set several number of epochs (20 , 50 , 100) on the training process. However, we can easily get almost **99%+** accuracy within 20 or 30 epochs. Training on a single CPU, epochs size should be set within 2 or 3~4, accuracy almost **99%**.

---

Let's begin with importing packages and dataset.

In [20]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

# Load the data
import pandas as pd
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")

Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

print('Digit Counter:\n',Y_train.value_counts() , '\n')
print('-'*30)
print('Check for missing values on training set:\n',X_train.isnull().any().describe() , '\n')
print('-'*30)
print('Check for Missing values on test set:\n',test.isnull().any().describe() , '\n')
print('-'*30)

# Normalize the data
X_train = X_train / 255.0
test = test / 255.0

# Reshape image in 3 dimensions 
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# On hot encoding 
from keras.utils.np_utils import to_categorical 
Y_train = to_categorical(Y_train, num_classes = 10)

# Set the random seed
random_seed = 101

# Randomly split the data sets
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1,
                                                  random_state=random_seed)

# CNN-Keras Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(10, activation = "softmax"))


import keras
adamax = keras.optimizers.Adamax(lr=0.002, beta_1=0.9, 
                                 beta_2=0.999, epsilon=None, decay=0.0)

# Compile the model
model.compile(optimizer = adamax , 
              loss = "categorical_crossentropy", 
              metrics=["accuracy"])

# Set a learning rate annealer
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

# Using the Tensorboard callback of Keras. 
# https://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
tbCallBack = keras.callbacks.TensorBoard(log_dir='ConvNet/KerasGraph', histogram_freq=0, 
                                         write_graph=True, write_images=True)

epochs = 20 # Turn epochs to 20 ~ 50 (strong GPU) or 2 ~ 5 (strong CPU)
batch_size = 64

# Data Augmentation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

Digit Counter:
 1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64 

------------------------------
Check for missing values on training set:
 count       784
unique        1
top       False
freq        784
dtype: object 

------------------------------
Check for Missing values on test set:
 count       784
unique        1
top       False
freq        784
dtype: object 

------------------------------


In [21]:
%%time
# Fit the model
trained = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction, tbCallBack])

Epoch 1/20
590/590 [==============================] - 9s 15ms/step - loss: 0.6090 - acc: 0.8001 - val_loss: 0.0745 - val_acc: 0.9788
Epoch 2/20
590/590 [==============================] - 8s 14ms/step - loss: 0.1906 - acc: 0.9404 - val_loss: 0.1300 - val_acc: 0.9586
Epoch 3/20
590/590 [==============================] - 8s 13ms/step - loss: 0.1417 - acc: 0.9557 - val_loss: 0.0395 - val_acc: 0.9881
Epoch 4/20
590/590 [==============================] - 8s 13ms/step - loss: 0.1141 - acc: 0.9640 - val_loss: 0.0395 - val_acc: 0.9881
Epoch 5/20
590/590 [==============================] - 8s 14ms/step - loss: 0.0999 - acc: 0.9693 - val_loss: 0.0324 - val_acc: 0.9895
Epoch 6/20
590/590 [==============================] - 8s 14ms/step - loss: 0.0954 - acc: 0.9704 - val_loss: 0.0297 - val_acc: 0.9902
Epoch 7/20
590/590 [==============================] - 8s 14ms/step - loss: 0.0816 - acc: 0.9747 - val_loss: 0.0297 - val_acc: 0.9912
Epoch 8/20
590/590 [==============================] - 8s 13ms/step - 

In [22]:
loss, acc = model.evaluate(X_val, Y_val, verbose=0)
print("Final loss: {0:.5f}, Final accuracy: {1:.5f}".format(loss, acc))

Final loss: 0.01115, Final accuracy: 0.99595


In [23]:
# Look at confusion matrix 
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)


    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [24]:
# display error results 
# Errors are difference between predicted labels and true labels

errors = (Y_pred_classes - Y_true != 0)
Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2 
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-6:]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

## Saving Model and Weights
We can save the model in `json` and weights in a `hdf5` file format.

In [25]:
from keras.models import model_from_json

# serialize model to JSON
# the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()
print('Saving Model Into JSON.')
with open("ConvNet/KerasTrained_Model/trained_model.json", "w") as json_file:
    json_file.write(model_json)

# saving weights to HDF5
print('Saving Trained Model Weights Into HDF5.')
model.save_weights('ConvNet/KerasTrained_Model/trained_model_weights.h5')

Saving Model Into JSON.
Saving Trained Model Weights Into HDF5.


## Load Saved Model and Weights
To use the same trained model for further testing we can simply load the `hdf5` file and use it for the prediction of different data. To load the **weights**, we need first to build our **model** which we've already done.  

In [26]:
# load json and create model
json_file = open('ConvNet/KerasTrained_Model/trained_model.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("ConvNet/KerasTrained_Model/trained_model_weights.h5")
print("Loaded model from disk")

Loaded model from disk


## Check Reloaded Model

In [27]:
loaded_model.compile(optimizer = adamax , loss = "categorical_crossentropy", metrics=["accuracy"])
loss, acc = loaded_model.evaluate(X_val, Y_val, verbose=0)

print("Final loss: {0:.5f}, Final accuracy: {1:.5f}".format(loss, acc))

Final loss: 0.01115, Final accuracy: 0.99595


---

## Save Model + Weight Together - Pretrainede Model
Another saving technique is `model.save(filepath)`. This save function saves:

- The architecture of the model, allowing to re-create the model.
- The weights of the model.
- The training configuration (loss, optimizer).
- The state of the optimizer, allowing to resume training exactly where we left off.

In [28]:
# Same Model and Weight at a same time - no need to compile as we previously did
# where we manually save model and weight and loaded them separately.
model.save('ConvNet/KerasTrained_Model/pretrained_model.h5')
from keras.models import load_model
loaded_model = load_model('ConvNet/KerasTrained_Model/pretrained_model.h5')
print('Loaded model from disk.')

Loaded model from disk.


In [29]:
loss, acc = loaded_model.evaluate(X_val, Y_val, verbose=0)
print("Final loss: {0:.5f}, Final accuracy: {1:.5f}".format(loss, acc))

Final loss: 0.01115, Final accuracy: 0.99595


# Submission 
Create a submission file for kaggle.

In [30]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results, axis = 1)
results = pd.Series(results, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"),results],axis = 1)
submission.to_csv("ConvNet/submission.csv",index=False)